# Requirements

In [ ]:
%pip install transformers
%pip install torch
%pip install pickle5
%pip install mpld3
%pip install scikit-learn

# Load pretrained Bert model and tokenizer

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=model.to(device)
model.eval()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# open file and extract bert embedding function

In [ ]:
import pickle5 as pickle
from tqdm import tqdm 

def dump_pklfile(file, filepath, size):
	with open(filepath, "wb") as f:
		if (size == 0):
			pickle.dump((file), f)
			return
		if(size > 0):
			pickle.dump((file[:size]), f)
			return
		else:
			pickle.dump((file[size:]), f)
			return
    
def open_pklfile(filepath, size):
	with open(filepath, "rb") as f:
		if (size == 0):
			return pickle.load(f)
		return (pickle.load(f))[0:size]

def extract_bert_embeddings(word_list):
	#init for stacking embeddings
	embeddings = torch.empty(0, device=device)
	for word in tqdm(word_list):
		
		if (word in '___'):
			continue
		# Map the token strings to their vocabulary indeces.
		marked_text = "[CLS] " + word + " [SEP]"
		tokenized_text = tokenizer.tokenize(marked_text)
		
		# handling such as "wedding_dress"
		tokenized_text = [token for token in tokenized_text if token != '_']

		# Split the sentence into tokens.
		indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
		segments_ids = [1] * len(tokenized_text)

		# Convert inputs to PyTorch tensors
		tokens_tensor = torch.tensor([indexed_tokens], device=device)
		segments_tensors = torch.tensor([segments_ids], device=device)
		
		# Put the model in "evaluation" mode,meaning feed-forward operation.
		model.eval()

		#Run the text through BERT, get the output and collect all of the hidden states produced from all 12 layers.
		with torch.no_grad():
			outputs = model(tokens_tensor, segments_tensors)

			last_four_hidden_states = outputs[2][-4:]
			concated_hidden_states = torch.cat(last_four_hidden_states, dim = 2)

			first_last = torch.add(concated_hidden_states[:, 1], concated_hidden_states[:, -2])
			embeddings = torch.cat([embeddings, first_last])

	print(embeddings.shape)
	return torch.squeeze(embeddings)

# computing bert bias function

In [ ]:
import numpy as np
from torch import linalg as LA
import scipy.stats
import json 
import codecs

def normalize(wv):    
    # normalize vectors
    # norms = np.apply_along_axis(LA.norm, 1, wv)
    norms=LA.norm(wv, dim=1)
    wv = wv / norms[:, np.newaxis]
    return wv

# compute bias from bert with he-she
def compute_bias_by_projection_heshe(vocab, lim_wv, gender_word_embedding):
    print(lim_wv.shape)
    print(gender_word_embedding[0].shape)
    males = torch.tensordot(lim_wv, gender_word_embedding[0], dims=1)
    females = torch.tensordot(lim_wv, gender_word_embedding[1], dims=1)
    d = {}
    for w,m,f in zip(vocab, males, females):
        d[w] = m-f
    return d

# compute bias from bert with seat method
def compute_bias_by_projection_seat(vocab, lim_wv, gender_word_embedding):
    return

def extract_professions():
    professions = []
    with codecs.open('../data/lists/professions.json', 'r', 'utf-8') as f:
        professions_data = json.load(f)
    for item in professions_data:
        professions.append(item[0].strip())
    return professions

## gender biased words computed on bert (2016 words vocab)

In [ ]:
vocab = open_pklfile("../data/extracted/2016_restricted_vocab.pkl", 0)
lim_wv = extract_bert_embeddings(vocab)
dump_pklfile(lim_wv, "../data/extracted/2016_restricted_embeddings.pkl", 2500)
lim_wv = normalize(lim_wv)
w2i = {w: i for i, w in enumerate(vocab)}

In [ ]:
with open("../data/lists/male_word_file.txt", 'r') as f:
  male_words = [word.strip() for word in f.readlines()]

with open("../data/lists/female_word_file.txt", 'r') as f:
  female_words = [word.strip() for word in f.readlines()]

male_embeddings = extract_bert_embeddings(male_words)
female_embeddings = extract_bert_embeddings(female_words)

male_embedding = torch.sum(male_embeddings, dim = 0, keepdim = True)
female_embedding = torch.sum(female_embeddings, dim = 0, keepdim = True)

male_embeddings = normalize(male_embeddings)
female_embeddings = normalize(female_embeddings)

gender_word_embedding = torch.cat((male_embedding, female_embedding), 0)

gender_bias_all = compute_bias_by_projection_heshe(vocab, lim_wv, gender_word_embedding)

In [ ]:
import operator
sorted_g = sorted(gender_bias_all.items(), key=operator.itemgetter(1))
for w in sorted_g[:2500]:
    word2emb = {w[0]: lim_wv[w2i[w[0]]]}
dump_pklfile(sorted_g, "../data/extracted/bert_2016_female_2500.pkl", 2500)

sorted_g = sorted(gender_bias_all.items(), key=operator.itemgetter(1), reverse=True)
for w in sorted_g[:2500]:
    word2emb = {w[0]: lim_wv[w2i[w[0]]]}
dump_pklfile(sorted_g, "../data/extracted/bert_2016_male_2500.pkl", 2500)

## gender biased words computed on bert (2018 words vocab)

In [ ]:
vocab = open_pklfile("../data/extracted/2018_restricted_vocab.pkl", 0)
lim_wv = extract_bert_embeddings(vocab)
dump_pklfile(lim_wv, "../data/extracted/2018_restricted_embeddings.pkl", 2500)
lim_wv = normalize(lim_wv)
w2i = {w: i for i, w in enumerate(vocab)}

In [ ]:
with open("../data/lists/male_word_file.txt", 'r') as f:
  male_words = [word.strip() for word in f.readlines()]

with open("../data/lists/female_word_file.txt", 'r') as f:
  female_words = [word.strip() for word in f.readlines()]

male_embeddings = extract_bert_embeddings(male_words)
female_embeddings = extract_bert_embeddings(female_words)

male_embedding = torch.sum(male_embeddings, dim = 0, keepdim = True)
female_embedding = torch.sum(female_embeddings, dim = 0, keepdim = True)

male_embeddings = normalize(male_embeddings)
female_embeddings = normalize(female_embeddings)

gender_word_embedding = torch.cat((male_embedding, female_embedding), 0)

gender_bias_all = compute_bias_by_projection_heshe(vocab, lim_wv, gender_word_embedding)

In [ ]:
import operator
sorted_g = sorted(gender_bias_all.items(), key=operator.itemgetter(1))
for w in sorted_g[:2500]:
    word2emb = {w[0]: lim_wv[w2i[w[0]]]}
dump_pklfile(sorted_g, "../data/extracted/bert_2018_female_2500.pkl", 2500)

sorted_g = sorted(gender_bias_all.items(), key=operator.itemgetter(1), reverse=True)
for w in sorted_g[:2500]:
    word2emb = {w[0]: lim_wv[w2i[w[0]]]}
dump_pklfile(sorted_g, "../data/extracted/bert_2018_male_2500.pkl", 2500)

In [ ]:
with open("../data/lists/male_word_file.txt", 'r') as f:
  male_words = [word.strip() for word in f.readlines()]
male_word_embs = extract_bert_embeddings(male_words)
for i in range(len(male_words)):
  malew2emb = {male_words[i]: male_word_embs[i]}
dump_pklfile(male_words, "../data/extracted/male_word_file_embeddings.pkl",0 )


with open("../data/lists/female_word_file.txt", 'r') as f:
  female_words = [word.strip() for word in f.readlines()]
female_word_embs = extract_bert_embeddings(female_words)
for i in range(len(female_words)):
  femalew2emb = {female_words[i]: female_word_embs[i]}
dump_pklfile(male_words, "../data/extracted/female_word_file_embeddings.pkl", 0)